In [2]:
import pandas as pd
import random
import os
import numpy as np
from project import *
import seqlogo
import timeit
from collections import Counter

In [3]:
# This was used to perform some minor cleaning on data from the original data from the paper
#df = pd.read_excel("motif_data.xlsx" ,engine='openpyxl',header=2,usecols=range(0,9))
# df.to_csv("motif_clean.csv",index=False)

In [4]:
#Read In Data
df = pd.read_csv("motif_clean.csv",index_col=False)
#Rename column to shorter name
df.rename(columns={'DNAseq (+/- 50nts from peak center)':'Seq'}, inplace=True)

In [5]:
#What does the data look like?
df.head()

,Regulator,Gene,p-Value,Score,VPM,Peak Start (Fstart),Peak Stop (Rstop),Peak Center (Ccenter),Seq
0,Rv0020c,Rv0020c,0.008387,0.708045,747,24834,25442,25262.0,CGACTGGATGCCGTCGGCCGCCTCGCGGCGCAGCAGGGCTTCGACC...
1,Rv0022c,MT0031,0.000000,0.938136,371,31853,32123,31964.0,TTCTCGCGCTCGAGTGAGCAGAGCCCGCACCTCGTCGAGTTGCTGC...
2,Rv0022c,MT0124,0.006907,0.747817,42,139867,140092,139961.0,AATGACGCACGCTGATCGGGCTTCCTGCAGGAGAAGAACATGACCA...
3,Rv0022c,MT0407.1,0.002960,0.837341,42,476142,476370,476251.0,CCGGTAGGCGCCTGCCCAAAACACGGGTATTGGGTAAAGGCACGGG...
4,Rv0022c,MT1759,0.008387,0.710485,22,1946282,1946498,1946420.0,CGCGTGGTTCAACGGCACTGAGGATCGCAAATGACAGAAGCGTTGT...


In [6]:
cleanSeqs = clean_strings(df["Seq"].tolist())
zero_order_freq = zero_order_freq(cleanSeqs)
first_order_freq = first_order_freq(cleanSeqs)

In [10]:
def run_gibbs(seqs,k=9,back=None):
    return GibbsSampler(seqs,k=k,numSeq=len(seqs),seqLen=len(seqs[0]),itr=3000,background=back)

In [ ]:
tst = list(list(df.groupby(['Regulator'])["Seq"])[2][1])
import time
start_time = time.time()
ans = GibbsSampler(tst,k=9,numSeq=len(tst),seqLen=100,itr=5000)
print("--- %s seconds ---" % (time.time() - start_time))
ppm = seqlogo.Ppm(ans.T)
t = seqlogo.seqlogo(ppm, ic_scale = False, format = 'png', size = 'medium')
ppm.ic

In [8]:
reg_groups = df.groupby("Regulator")["Seq"]
regulators = [k for k,v in reg_groups if len(v) > 50]

In [12]:
x = clean_strings(reg_groups.get_group("Rv0022c").tolist())

In [19]:
t = run_gibbs(x,back=fs)

In [28]:
t[:,3].sum()

1.0

In [15]:
zr = get_background(x,9,zero_order_freq,0)
fs = get_background(x,9,first_order_freq,1)

In [ ]:
zr.shape

In [33]:
def get_pwm():
    reg_groups = df.groupby("Regulator")["Seq"]
    regulators = [k for k,v in reg_groups if len(v) > 50]
    print("Creating PWMs for ",str(len(regulators))," regulators with lengths greater than 50")
    dct = {}
    for reg,n in zip(regulators,range(len(regulators))):
        print("Creating for ",str(reg)," ,number : ",str(n)," num seq : ",len(reg_groups.get_group(reg)))
        cleanseq = clean_strings(reg_groups.get_group(reg).tolist())
        zr = get_background(cleanseq,9,zero_order_freq,0)
        fs = get_background(cleanseq,9,first_order_freq,1)
        dct[reg] = {
            "no_bg" : run_gibbs(cleanseq,back=None),
            "zero_bg" : run_gibbs(cleanseq,back=zr),
            "one_bg" : run_gibbs(cleanseq,back=fs)
        }
    return dct

In [ ]:
res = get_pwm()

Creating PWMs for  51  with lengths greater than 50
Creating for  Rv0022c  ,number :  0  num seq :  218
Creating for  Rv0023  ,number :  1  num seq :  347
Creating for  Rv0047c  ,number :  2  num seq :  634
Creating for  Rv0081  ,number :  3  num seq :  856
Creating for  Rv0135c  ,number :  4  num seq :  439
Creating for  Rv0273c  ,number :  5  num seq :  80
Creating for  Rv0302  ,number :  6  num seq :  285
Creating for  Rv0324  ,number :  7  num seq :  437
